# csv文件读取

In [7]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [28]:
def csvread(filelist):
    """
    读取csv文件
    :param   filelist：文件路径+名字的列表
    :return   读取的内容
    """
    
    #1.构造文件队列
    file_queue = tf.train.string_input_producer(filelist)
    
    #2.构造csv阅读器读取队列数据（默认：行）
    reader = tf.TextLineReader()
    key,value = reader.read(file_queue)
    
    #3.对每行内容解码
        #1.record_defaults指定每一样本的每一列的类型（int,float...）
        #2.指定默认值
    records = [ ["None"],["None"] ]
    #有几列，有几个返回值
    example,label = tf.decode_csv(value,record_defaults=records)
    
    #4.批处理读取多个数据
    #批处理大小，跟队列，数据的数量没有影响，只决定这批次取多少数据
    example_batch,label_batch = tf.train.batch(
        [example,label],
        batch_size=20,
        num_threads=1,
        capacity=9
    )
    
    return example_batch,label_batch 
    
    
if __name__ == '__main__':
    
    #找到文件，放入列表
    file_name = os.listdir("./csvdata/")
    filelist = [os.path.join('./csvdata/',file) for file in file_name]
    
    example_batch,label_batch = csvread(filelist)
    
    #开始会话运行结果
    with tf.Session() as sess:
        
        #定义一个线程协调器
        coord = tf.train.Coordinator()
        
        #开启读文件的线程
        threads = tf.train.start_queue_runners(sess,coord=coord)
        
        #打印读取的内容
        print(sess.run([example_batch,label_batch]))
        
        #回收子线程
        coord.request_stop()
        coord.join(threads)

[array([b'See1', b'See2', b'See3', b'Alpha1', b'Alpha2', b'Alpha3',
       b'Bee1', b'Bee2', b'Bee3'], dtype=object), array([b'C1', b'C2', b'C3', b'A1', b'A2', b'A3', b'B1', b'B2', b'B3'],
      dtype=object)]


# 图片读取

In [ ]:
def picread(filelist):
    """
    读取狗图片并转换成张量
    ：param  filelist：文件路径+名字列表
    ：return 每张图片的张量
    """
    
    #1.构造文件队列
    file_queue = tf.train.string_input_producer(filelist)
    
    #2.构造阅读器去读取图片内容（默认读取一张图片）
    reader = tf.WholeFileReader()
    key,value = reader.read(file_queue)
    
    #3.对读取的图片数据进行解码
    image = tf.image.decode_jpeg(value)
    
    #4.处理图片（统一大小）
    image_resize = tf.image.resize_images(image,[200,200])
    
    #把样本的形状固定（设置通道数）,在批处理时要求所有数据形状必须定义
    image_resize.set_shape([200,200,3])
    
    #5.批处理
    image_batch = tf.train.batch(
        [image_resize],
        batch_size=20,
        num_threads=1,
        capacity=1
    )
    
    return image_batch

if __name__ == '__main__':    
    #找到文件，放入列表
    file_name = os.listdir("./picdata/")
    filelist = [os.path.join('./picdata/',file) for fi le in file_name]
    
    image_batch = picread(filelist)
    
    #开始会话运行结果
    with tf.Session() as sess:
        
        #定义一个线程协调器
        coord = tf.train.Coordinator()
        
        #开启读文件的线程
        threads = tf.train.start_queue_runners(sess,coord=coord)
        
        #打印读取的内容
        print(sess.run([image_batch]))
        
        #回收子线程
        coord.request_stop()
        coord.join(threads)

# 二进制文件读取

In [ ]:
class CifarRead(object):
    """
    完成读取二进制图片，写入tfrecords，读取tfrecords
    """
    
    def __init__(self,filelist):
        #文件列表
        self.file_list = filelist
        
        #定义读取图片的属性
        self.height = 32
        self.width = 32
        self.channel = 3
        
        #二进制每张图片的字节
        self.label_bytes = 1
        self.image_bytes = self.height * self.width * self.channel
        self.bytes = self.label_bytes + self.image_bytes
    
    def read_and_edcode(self):
        
        #1.构造文件队列
        file_queue = tf.train.string_input_producer(slef.file_list)
        
        #2.构造二进制文件读取器，读取内容(每个样本的字节数)
        reader = tf.FixedLengthRecordReader(self.bytes)
        
        key,value = reader.read(file_queue)
        
        #3.二进制内容解码
        label_image = tf.decode_raw(value,tf.uint8)
        
        #4.分割出图片和标签数据，切割特征值和目标值 
        label = tf.cast(
                    tf.slice(
                            label_image,
                            [0],
                            [self.label_bytes]
                        ),
                    tf.int32
                )
        image = tf.slice(label_image,[self.label_bytes],[self.image_bytes])
        
        #5.可对图片的特征数据进行形状的改变 [3072] --> [32,32,3]
        image_reshape = tf.reshape(image,[self.height,self.width,self.channel])
        
        #6.批处理数据
        image_batch,label_batch = tf.train.batch([image_reshape,label],batch_size=10,num_threads=1,capacity=10)
        
        return image_batch,label_batch
    

if __name__ == '__main__':    
    #找到文件，放入列表
    file_name = os.listdir("./cifar10/")
    filelist = [os.path.join('./cifar10/',file) for fi le in file_name if file[-3:]=='bin']
    
    cf = CifarRead(filelist)
    image_batch,label_batch = cf.read_and_edcode()
    
    #开始会话运行结果
    with tf.Session() as sess:
        
        #定义一个线程协调器
        coord = tf.train.Coordinator()
        
        #开启读文件的线程
        threads = tf.train.start_queue_runners(sess,coord=coord)
        
        #打印读取的内容
        print(sess.run([image_batch,label_batch]))
        
        #回收子线程
        coord.request_stop()
        coord.join(threads)

# TFRecords读取

In [ ]:
lass CifarRead(object):
    """
    完成读取二进制图片，写入tfrecords，读取tfrecords
    """
    
    def __init__(self,filelist):
        #文件列表
        self.file_list = filelist
        
        #定义读取图片的属性
        self.height = 32
        self.width = 32
        self.channel = 3
        
        #二进制每张图片的字节
        self.label_bytes = 1
        self.image_bytes = self.height * self.width * self.channel
        self.bytes = self.label_bytes + self.image_bytes
    
    def read_and_edcode(self):
        
        #1.构造文件队列
        file_queue = tf.train.string_input_producer(slef.file_list)
        
        #2.构造二进制文件读取器，读取内容(每个样本的字节数)
        reader = tf.FixedLengthRecordReader(self.bytes)
        
        key,value = reader.read(file_queue)
        
        #3.二进制内容解码
        label_image = tf.decode_raw(value,tf.uint8)
        
        #4.分割出图片和标签数据，切割特征值和目标值 
        label = tf.cast(tf.slice(label_image,[0],[self.label_bytes]),tf.int32)
        image = tf.slice(label_image,[self.label_bytes],[self.image_bytes])
        
        #5.可对图片的特征数据进行形状的改变 [3072] --> [32,32,3]
        image_reshape = tf.reshape(image,[self.height,self.width,self.channel])
        
        #6.批处理数据
        image_batch,label_batch = tf.train.batch([image_reshape,label],batch_size=10,num_threads=1,capacity=10)
        
        return image_batch,label_batch
    
    def write_ro_tfrecords(self.image_batch,label_batch):
        """
        将图片的特征值和目标值存进tfrecords
        :param  image_batch：10张图片的特征值
        :param  label_batch：10张图片的目标值
        :return: None
        """
        
        #1.建立TFRecords存储器
        # cifar.tfrecords
        writer = tf.python_io.TFRecords('./TFRecords/')
        
        #2.循环将所有样本写入文件，每张图片样本都要构造example协议
        for i in range(10):
            #取出第i个图片数据的特征值和目标值
            image = image_batch[i].eval().tostring()
            label = label_batch[i].eval()[0]
            
            #构造一个样本的example
            example = tf.train.Example(
                features=tf.train.Features(
                    feature={
                        "image":tf.train.Feature(
                            bytes_list=tf.train.BytesList(value=[image])
                        ),
                        "label":tf.train.Feature(
                            int64_list=tf.train.Int64List(value=[label])
                        ),
                    }
                )
            )
            
            #写入单独的样本
            writer.write(example.SerializeToString())
            
            #关闭
            writer.close()
    
        return None
    
    def read_from_tfrecords(self):
        
        """
        读取tfrecords文件
        """
        
        #1.构造文件队列
        file_queue = tf.train.string_input_producer("./tfrecords/")
        
        #2.构造文件阅读器，读取内容Example,value是一个样本的序列化example
        reader = tf.TFRecordReader()
        key, value = reader.read(file_queue)
        
        #3.解码example
        features = tf.parse_single_eample(value,features={
            "image":tf.FixedLenFeature([],tf.string),
            "label":tf.FixedLenFeature([],tf.int64),
        })
        
        
        
        #4.解码内容，如果读取的内容格式是String需要解码，如果int64，float32不需要解码
        image = tf.decode_raw(features['image'],tf.uint8)
        
        #固定图片的形状，方便于批处理
        image_reshape = tf.reshape(image,[self.height,slef.width,self.channel])
        
        label = tf.cast(features['label'],tf.int32)
        
        #5.进行批处理
        image_batch,label_batch = tf.train.batch([image_reshape,label],batch_size=10,num_threads=1,capacity=10)
        
        return image_batch,label_batch
    
    
    
    
    
if __name__ == '__main__':    
    #找到文件，放入列表
    file_name = os.listdir("./cifar10/")
    filelist = [os.path.join('./cifar10/',file) for fi le in file_name if file[-3:]=='bin']
    
    cf = CifarRead(filelist)
    
    #读取tfrecords文件
    image_batch,label_batch = cf.read_from_tfrecords()
    
    #写入tfrecords文件
    #image_batch,label_batch = cf.read_and_edcode()
    
    #开始会话运行结果
    with tf.Session() as sess:
        
        #定义一个线程协调器
        coord = tf.train.Coordinator()
        
        #开启读文件的线程
        threads = tf.train.start_queue_runners(sess,coord=coord)
        
        #存入tfrecord文件
        cf.write_ro_tfrecords(image_batch,label_batch)
        
        
        #打印读取的内容
        print(sess.run([image_batch,label_batch]))
        
        #回收子线程
        coord.request_stop()
        coord.join(threads)